In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
 

In [2]:
md = pd.read_excel(io='CCLE metabolomics dataset.xlsx',sheet_name="All")
mt = md.drop(['Tissue', 'Medium','Culture'], axis=1)
hm = pd.read_csv('GCP_proteomics_remapped.csv')
merge_tb = mt.merge(hm,how='inner',left_on='CCL', right_on='Cell Line')
MET = merge_tb.iloc[:,1:226]
GCP = merge_tb.iloc[:,227:269]
print(MET.head())
print(GCP.head())


   2-aminoadipate  3-phosphoglycerate  Alpha-glycerophosphate  4-pyridoxate  \
0        6.112727            6.034198                5.896896      6.000532   
1        5.577413            5.727045                5.111468      6.073250   
2        5.886398            5.574881                5.541259      5.848375   
3        5.770030            6.099229                6.233259      5.543495   
4        5.480683            5.469742                6.509397      6.251005   

   Aconitate   Adenine   Adipate  Alpha-ketoglutarate       AMP   Citrate  \
0   5.513618  5.868529  5.977177             5.693074  5.923737  5.641242   
1   5.802494  5.824473  5.888821             5.768379  5.760784  5.914742   
2   5.665026  5.875548  5.894904             5.839640  5.742613  5.570208   
3   5.767759  6.155905  6.111148             5.949481  6.342703  6.054781   
4   5.190578  5.897085  6.148333             5.607481  5.871600  5.128463   

   ...  C56:8 TAG  C56:7 TAG  C56:6 TAG  C56:5 TAG  C56:4 TAG 

In [3]:
# GCP.to_numpy()
# MET.to_numpy()
GCP = np.nan_to_num(GCP, nan=0)
MET = np.nan_to_num(MET, nan=0)
Xtrain, Xtest, Ytrain, Ytest =train_test_split(GCP, MET, test_size=0.3, random_state=0)

# print(GCP.shape)
# print(Xtrain)
print(Xtest.shape)
# print(MET.shape)
print(Ytrain.shape)
# print(Ytest.shape)

(259, 42)
(604, 225)


In [4]:

GCP2MET_models = []
for i in range(Ytrain.shape[1]):
    mdl_G2M = LinearRegression().fit(Xtrain, Ytrain[:, i])
    GCP2MET_models.append(mdl_G2M)

#     GCP2MET_models.append(mdl_G2M.coef_)
#     Ypred = mdl_G2M.coef_ * Xtest
print(GCP2MET_models)
# print(len(GCP2MET_models))

MET2GCP_models = []

for j in range(Xtrain.shape[1]):
    mdl_M2G = LinearRegression().fit(Ytrain, Xtrain[:,j])
    MET2GCP_models.append(mdl_M2G)

#     MET2GCP_models.append(mdl_M2G.coef_)
#     print(MET2GCP_models)
# print(len(MET2GCP_models))

[LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(),

In [5]:
def evaluate_models(models, Xtest, Ytest):
    """
    evaluate_models returns results from the model predictions, including the pearson
    correlation coefficient, p-Values, and MSE.

    :param models:         A list of scikit-learn model objects.
    :param Xtest:          A numpy array or pandas dataframe containing validation set input data.
    :param Ytest:          A numpy array or pandas dataframe containing validation set output data.
    :return pred_resul:    A dictionary containing the final MSE, pValue, or rValue.
    
    """
    
    predictions = []
    rValue = list()
    pValue = list()
    MSE = list()
    for i in range(len(models)):
        mdl = models[i]
        Ypred = mdl.predict(Xtest)
        predictions.append(Ypred)
        
        r, pvalue = pearsonr(Ypred, Ytest[:, i])
        rValue.append(r)
        pValue.append(pvalue)
        
        mse = mean_squared_error(Ypred, Ytest[:, i])
        MSE.append(mse)
           

    df_MSE = pd.DataFrame(MSE)
    df_pValue = pd.DataFrame(pValue)
    df_rValue = pd.DataFrame(rValue)
    
    return df_MSE, df_pValue, df_rValue

mse1 = evaluate_models(GCP2MET_models, Xtest, Ytest)




In [6]:
mse1,p1,pearson1 = evaluate_models(GCP2MET_models, Xtest, Ytest)
print(mse1)

            0
0    0.105660
1    0.113761
2    0.211453
3    0.124643
4    0.102915
..        ...
220  0.157474
221  0.174273
222  0.269746
223  0.353366
224  0.266817

[225 rows x 1 columns]


In [7]:

kf = KFold(n_splits=3)

KFold(n_splits=3, random_state=None, shuffle=True)
mse_result = pd.DataFrame()
pvalue_result = pd.DataFrame()
rvalue_result = pd.DataFrame()
for train_index, test_index in kf.split(Xtrain, Ytrain):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = Xtrain[train_index], Xtrain[test_index]
    y_train, y_test = Ytrain[train_index], Ytrain[test_index]
    v_GCP2MET_models = []
    final_metrics = []
    for i in range(y_train.shape[1]):
        mdl_G2M = LinearRegression().fit(X_train, y_train[:, i])
        v_GCP2MET_models.append(mdl_G2M)
        df_MSE, df_pValue, df_rValue = evaluate_models(v_GCP2MET_models, X_test, y_test)
#     print("mse:_")
#     print(df_MSE)
    mse_result = pd.concat([mse_result,df_MSE],axis = 1)
    pvalue_result = pd.concat([pvalue_result,df_pValue],axis = 1)
    rvalue_result = pd.concat([rvalue_result,df_rValue],axis = 1)

    
mse_result.columns = ["mse1","mse2","mse3"]
mse_result = mse_result.T
print(mse_result)
       
    

# MET2GCP_models = []

# for j in range(Xtrain.shape[1]):
#     mdl_M2G = LinearRegression().fit(Ytrain, Xtrain[:,j])
#     MET2GCP_models.append(mdl_M2G)


           0         1         2         3         4         5         6    \
mse1  0.098891  0.094329  0.319502  0.113154  0.105149  0.133179  0.045786   
mse2  0.123628  0.103319  0.335725  0.139940  0.102402  0.140503  0.043725   
mse3  0.099345  0.103648  0.278150  0.128813  0.099365  0.136939  0.038375   

           7         8         9    ...       215       216       217  \
mse1  0.080428  0.250379  0.112139  ...  0.145670  0.202951  0.192299   
mse2  0.064761  0.253493  0.116281  ...  0.101652  0.203684  0.191712   
mse3  0.067909  0.191660  0.102278  ...  0.124050  0.197997  0.183160   

           218       219       220       221       222       223       224  
mse1  0.224030  0.169372  0.142034  0.170940  0.298064  0.340335  0.290046  
mse2  0.250278  0.189576  0.166025  0.201022  0.316814  0.396047  0.300359  
mse3  0.250604  0.200674  0.176717  0.198773  0.277209  0.326478  0.239974  

[3 rows x 225 columns]


In [8]:
minMSE = mse_result.idxmin()
print(minMSE)

print(mse_result)

0      mse1
1      mse1
2      mse3
3      mse1
4      mse3
       ... 
220    mse1
221    mse1
222    mse3
223    mse3
224    mse3
Length: 225, dtype: object
           0         1         2         3         4         5         6    \
mse1  0.098891  0.094329  0.319502  0.113154  0.105149  0.133179  0.045786   
mse2  0.123628  0.103319  0.335725  0.139940  0.102402  0.140503  0.043725   
mse3  0.099345  0.103648  0.278150  0.128813  0.099365  0.136939  0.038375   

           7         8         9    ...       215       216       217  \
mse1  0.080428  0.250379  0.112139  ...  0.145670  0.202951  0.192299   
mse2  0.064761  0.253493  0.116281  ...  0.101652  0.203684  0.191712   
mse3  0.067909  0.191660  0.102278  ...  0.124050  0.197997  0.183160   

           218       219       220       221       222       223       224  
mse1  0.224030  0.169372  0.142034  0.170940  0.298064  0.340335  0.290046  
mse2  0.250278  0.189576  0.166025  0.201022  0.316814  0.396047  0.300359  
mse3

In [9]:
# pvalue_result.columns = ["p1","p2","p3"]

# print(pvalue_result)

In [10]:
# rvalue_result.columns = ["r1","r2","r3"]
# print(rvalue_result)

In [11]:
for index in mse_result:
    print(mse_result[index]['mse1'])

0.09889121752201889
0.0943285422996137
0.31950231427523035
0.11315431881789152
0.10514939486950832
0.13317884818626838
0.04578579033959973
0.08042755920071351
0.2503789550024557
0.11213885851416264
0.12186793393342182
0.20190726264968384
0.3509429787253682
0.32541594919029554
0.08528650803056174
0.13763564355567962
0.08012526888079799
0.13253482205965422
0.19291412892579957
0.0559227796093987
0.06537129352056005
0.17886043510849625
0.1560246615350091
0.2222381669405133
0.215978795458972
0.11375716423000684
0.47815794583899096
0.11705687793958357
0.20353171323770963
0.07199777812843675
0.2287430673356061
0.08970158475257362
0.1299459584882021
0.1418574531686212
0.05665781864690379
0.18153708419971948
0.11706498897176869
0.17650933099444802
0.12781678968324506
0.07278866431745934
0.14995104442276785
0.160340673316442
0.25237900246603945
0.1960886414092957
0.14236544878279636
0.2003490583692743
0.3045762606239231
0.16073430871816632
0.13546511927557014
0.06190897430275994
0.86560106274065

In [12]:
mse_min = pd.DataFrame()
MSE_min = list()
for index in mse_result:
    v = mse_result[index][minMSE[index]]
    MSE_min.append(v)
#     print(v)
mse_min = pd.DataFrame(MSE_min)
print(mse_min)


            0
0    0.098891
1    0.094329
2    0.278150
3    0.113154
4    0.099365
..        ...
220  0.142034
221  0.170940
222  0.277209
223  0.326478
224  0.239974

[225 rows x 1 columns]


In [13]:
raw = mse1.join(mse_min, lsuffix='GCP2MET_models', rsuffix='v_GCP2MET_models')
print(raw)

     0GCP2MET_models  0v_GCP2MET_models
0           0.105660           0.098891
1           0.113761           0.094329
2           0.211453           0.278150
3           0.124643           0.113154
4           0.102915           0.099365
..               ...                ...
220         0.157474           0.142034
221         0.174273           0.170940
222         0.269746           0.277209
223         0.353366           0.326478
224         0.266817           0.239974

[225 rows x 2 columns]


In [14]:
GCP2MET_models
v_GCP2MET_models
new = list()
for index, row in raw.iterrows():
    if row['0GCP2MET_models'] < row['0v_GCP2MET_models']:
        
        new.append(GCP2MET_models[index])
    else:
        new.append(v_GCP2MET_models[index])
        
    
print(new)

[LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(), LinearRegression(),

In [15]:
# mse_result,p_result,pearson_result = evaluate_models(new, Xtest, Ytest)
# print(mse_result)